# 1. Collecting (데이터 수집 및 클렌징)

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.5 MB/s eta 0:00:00


In [2]:
!pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn

In [3]:
# 모듈 임포트 선언
import os
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re, unicodedata
from string import whitespace
from google.colab import drive

import time
from multiprocessing import Pool

from konlpy.tag import Okt
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim
from sklearn.model_selection import KFold

from gensim.models import CoherenceModel

AttributeError: ignored

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


# 파일 경로 지정
os.chdir('/content/drive/MyDrive/Project_Tesla')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 네이버뉴스 1개 크롤링
def news(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_element = soup.select_one('h2#title_area')
    title = title_element.text if title_element else None
    date_element = soup.select_one('span.media_end_head_info_datestamp_time')
    date = date_element.get('data-date-time') if date_element else None
    content_element = soup.select_one('article#dic_area')
    content = content_element.text.strip() if content_element else None
    return {
        'title': title,
        'date': date,
        'content': content
    }


# 네이버뉴스 페이지+원하는 날짜 크롤링
def news_list(keyword, startdate, enddate):
    li = []
    h = {'User-Agent': '...',
         'Referer': '...',
         'cookie': '...'}
    for d in pd.date_range(startdate, enddate):
        str_d = d.strftime("%Y.%m.%d")
        page = 1
        print(str_d)
        while True:
            start = (page - 1) * 10 + 1
            print(page)
            URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={0}&sort=2&photo=0&field=0&pd=3&ds={1}&de={2}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{3}to{4},a:all&start={5}".format(keyword, str_d, str_d, str_d.replace(".", ""), str_d.replace(".", ""), start)
            res = requests.get(URL, headers=h)
            soup = BeautifulSoup(res.text, "html.parser")
            if soup.select_one(".api_noresult_wrap"):
                break
            news_list = soup.select("ul.list_news li")
            for item in news_list:
                if len(item.select("div.info_group a")) == 2:
                    li.append(news(item.select("div.info_group a")[1]['href']))
            page = page + 1
    return pd.DataFrame(li, columns=['title', 'date', 'content'])


# 바이라인 제거
def clean_byline(text):
    # 바이라인
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 데이터프레임 전처리를 병렬적으로 처리

def work_func(x):
    print("value %s is in PID : %s" % (x, os.getpid()))
    time.sleep(1)
    return x**5

def main():
    start = int(time.time())
    num_cores = 4
    pool = Pool(num_cores)
    print(pool.map(work_func, range(1,13)))
    print("***run time(sec) :", int(time.time()) - start)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


value 3 is in PID : 89876value 1 is in PID : 89875value 2 is in PID : 89878
value 4 is in PID : 89877


value 5 is in PID : 89876value 6 is in PID : 89875value 8 is in PID : 89877value 7 is in PID : 89878



value 10 is in PID : 89875value 11 is in PID : 89878value 9 is in PID : 89876

value 12 is in PID : 89877

[1, 32, 243, 1024, 3125, 7776, 16807, 32768, 59049, 100000, 161051, 248832]
***run time(sec) : 3


In [ ]:
# 크롤링할 데이터 (키워드, 시작날짜, 종료날짜)
result_df = news_list('테슬라', '2022.11.01', '2022.11.30')

In [ ]:
# 크롤링한 데이터 불러오기
result_df = pd.read_csv('/content/drive/MyDrive/Project_Tesla/data/[2022-Jan]news_data_cleansing.csv', dtype='object')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 크롤링 데이터, 데이터 프레임에 저장 및 필요없는 column 삭제
df = pd.DataFrame(result_df)
df['content'] = df['content'].fillna('').astype(str).map(clean_byline)


# 유니코드 문자 전처리 및 정규 표현식 사용
pattern_whitespace = re.compile(f'[{whitespace}]+')
df['content'] = df['content'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


# 클렌징 데이터 csv 파일로 저장
df.to_csv('[2022-Nov]news_data_cleansing.csv', index=False, encoding='utf-8-sig')

# 2. Preprocessing (형태소 분석, 불용어 처리)

- 뉴스 데이터 병합 모드 추가하기
- OKt -> mecab으로 수정하기
- vscode로도 구현하기

In [ ]:
!pip install konlpy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# csv 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/Project_Tesla/data/[2022-Jan]news_data_cleansing.csv')


# 형태소 분석기 초기화
okt = Okt()


# 텍스트 데이터를 리스트로 변환
documents = df['content'].tolist()


# 불용어 리스트 정의
stop_words = ["것", "수", "이", "그", "를", "를", "등", "과", "에", "가", '때', '의', '및', '월', '위', '일', '억', '년', '원', '지난해', '를', '것', '등','차','올해','챗'
              '위', '가', '조', '의', '및','약','수','주','기자','만','이','중','말','마하','미']



# 각 문서를 형태소 분석 및 토큰화하고 불용어 제거
tokenized_documents = []
for document in documents:
    # 형태소 분석 수행 후 명사만 선택 (원하는 형태소 선택 가능)
    tokens = [word for word, pos in okt.pos(str(document)) if pos in ['Noun'] and word not in stop_words]
    tokenized_documents.append(tokens)


# 사전 (Dictionary) 생성
dictionary = corpora.Dictionary(tokenized_documents)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 3. Analysis (토픽 모델링, LASSO 회귀분석)

In [ ]:
# #엑셀 파일 병합하기
# filepath = f'/content/drive/MyDrive/Project_Tesla/data'

# #파일 경로
# file_list = os.listdir(filepath)

# #경로에 있는 모든 csv파일 리스트 불려오기
# file_list_csv = [file for file in file_list if file.endswith('csv')]

# #저장할 dataframe
# merged_df = pd.DataFrame()

# for file in file_list_csv:
#   #read_excel기능으로 파일 읽음
#   df = pd.read_csv(filepath + file, dtype='object')
#   merged_df = merged_df.append(df)

# #병합엑셀 파일 저장
# merged_df.to_csv("/content/drive/MyDrive/Project_Tesla/[2022]merge_news_data.csv",index=False, encoding='utf-8-sig')
# merged_df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Define a function to calculate perplexity
def calculate_perplexity(lda_model, corpus):

    perplexity = lda_model.log_perplexity(corpus)
    return perplexity

# Ensure that all elements in documents are strings
documents = [str(doc) for doc in documents]

# 교차 검증을 위한 접기 수를 정의
num_folds = 5

# 테스트할 알파 값의 범위를 정의
alpha_values = [0.01, 0.1, 0.2, 0.5, 1.0]

# 알파 튜닝을 위해 k-겹 교차 검증을 수행
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for alpha in alpha_values:
    # 교차 검증 결과를 저장하기 위해 목록을 초기화
    cv_results = []

    for train_index, test_index in kf.split(documents):
        train_data = [documents[i] for i in train_index]
        test_data = [documents[i] for i in test_index]

        # train_data 및 test_data에 대한 전처리(정리, 토큰화 등)
        # 훈련 데이터로부터 사전과 말뭉치를 생성
        tokenized_train_data = [okt.nouns(text) for text in train_data]
        dictionary = corpora.Dictionary(tokenized_train_data)
        corpus = [dictionary.doc2bow(text) for text in train_data]

        # 현재 알파 값으로 LDA 모델을 훈련
        lda_model = LdaModel(corpus, num_topics=30, id2word=dictionary, passes=15, alpha=alpha)

        # 테스트 데이터에 대한 모델을 평가(예: 복잡성 또는 기타 측정항목 사용)
         # 이것을 평가 코드로 바꿈
         # 예: perplexity = lda_model.log_perplexity(corpus_test)

        # 평가 결과를 결과 목록에 추가
        cv_results.append(perplexity)

    # 평가 결과의 평균과 표준편차를 계산
    mean_perplexity = sum(cv_results) / len(cv_results)
    std_perplexity = (sum((x - mean_perplexity) ** 2 for x in cv_results) / len(cv_results)) ** 0.5


    print(f"Alpha = {alpha}, Mean Perplexity = {mean_perplexity}, Std Perplexity = {std_perplexity}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: ignored

In [ ]:
# Tfidf 모델 생성
tfidf = TfidfModel(dictionary=dictionary)
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_documents]

# LDA 모델 출력
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic #{idx}: {topic}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic #0: 0.037*"자율" + 0.030*"주행" + 0.016*"기능" + 0.015*"기술" + 0.015*"차량" + 0.012*"자동차" + 0.012*"소프트웨어" + 0.011*"테슬라" + 0.011*"운전자" + 0.010*"시스템"
Topic #1: 0.085*"배터리" + 0.032*"전기차" + 0.020*"에너지" + 0.018*"생산" + 0.015*"중국" + 0.014*"시장" + 0.013*"솔루션" + 0.011*"업체" + 0.010*"테슬라" + 0.010*"공급"
Topic #2: 0.034*"금리" + 0.027*"인상" + 0.024*"준" + 0.013*"긴축" + 0.012*"미국" + 0.012*"하락" + 0.011*"전망" + 0.011*"시장" + 0.010*"파월" + 0.010*"증시"
Topic #3: 0.061*"코인" + 0.041*"도지" + 0.026*"머스크" + 0.024*"화폐" + 0.022*"맥도날드" + 0.019*"결제" + 0.016*"트위터" + 0.016*"테슬라" + 0.012*"가격" + 0.012*"가상"
Topic #4: 0.035*"재산" + 0.034*"세계" + 0.026*"버핏" + 0.024*"달러" + 0.021*"자산" + 0.019*"감소" + 0.016*"부자" + 0.015*"창업" + 0.015*"테슬라" + 0.014*"머스크"
Topic #5: 0.042*"코스피" + 0.025*"외국인" + 0.023*"코스닥" + 0.019*"개인" + 0.017*"기관" + 0.017*"선" + 0.016*"카카오" + 0.016*"매도" + 0.015*"부회장" + 0.015*"전자"
Topic #6: 0.030*"머스크" + 0.027*"테슬라" + 0.022*"스톡옵션" + 0.016*"니켈" + 0.015*"세금" + 0.013*"달러" + 0.012*"흑연" + 0.011*"행사" + 0.010*"가격" + 0.010*"개"
Topic #7:

In [ ]:
!pip install pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 엑셀 파일 경로, 데이터 프레임으로 읽기
excel_file_path = r'/content/drive/MyDrive/Project_Tesla/stockdata/TSLA_DATA_2022_daily.csv'
df_volume = pd.read_csv(excel_file_path)


# 거래량 열에서 'M'을 제거하고 숫자로 변환
df_volume['거래량'] = pd.to_numeric(df_volume['거래량'].str.replace('M', ''))
# 거래량 열의 값을 모두 더함
total_volume = df_volume['거래량'].sum()
print(total_volume)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


21826.16


In [ ]:
n_samples = len(df_volume)  # 데이터 샘플
n_features = len(lda_model.get_document_topics(corpus[0]))  # 토픽 수

# X(거래량) 및 Y(주제)에 대한 합성 데이터 생성
X = df_volume['거래량'].values.reshape(-1, 1)
Y = np.random.rand(n_samples, n_features) # 실제 토픽 모델링 데이터로 대체

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# LASSO 회귀 수행, 거래량을 기반으로 Y 값을 예측
lasso = Lasso(alpha=0.01)  # 알파 값을 조정
lasso.fit(X_train, Y_train)
predicted_Y = lasso.predict(X_test)


# LASSO 회귀 결과를 주제 모델링 데이터와 결합하여 새 DataFrame에 저장
combined_data = pd.DataFrame({'TradingVolume': X_test.flatten()})  # 거래량과 예상 토픽 값
for i in range(n_features):
    combined_data[f'Topic_{i}'] = predicted_Y[:, i]  # 예측된 Y 값을 토픽 모델링 데이터에 추가


# 주제와 주식 데이터 간의 관계를 시각화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
